In [ ]:
from Extern import preproc_load_raw
from Extern import tmsi_poly5reader
from matplotlib import pyplot as plt
import scipy
from scipy.signal import spectrogram, hann, butter, filtfilt
from importlib import reload
import dat_preproc
import pandas as pd
import numpy as np
import os
import mne
from utils import find_folders

#### Load and adjust Externalized 

In [ ]:
dat_path = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Extern_Trial\\572NR53_MedOn1_StimOff_Rest1 - 20230915T140107'

raw_file = os.path.join(
    dat_path,
    '572NR53_MedOn1_StimOff_Rest1-20230915T140107.DATA.Poly5'
)

raw = tmsi_poly5reader.Poly5Reader(raw_file)
raw = raw.read_data_MNE()

tstamps_sec = (1 / raw.info['sfreq']) * np.arange(raw.n_times)
new_ch_name = 'TimeinSec'
new_data_2d = tstamps_sec.reshape(1,-1)
new_info = mne.create_info(['Time_Sec'], raw.info['sfreq'], ch_types=['misc'])

# Create a RawArray object for the new channel
new_raw_array = mne.io.RawArray(new_data_2d, new_info)

# Add the new channel to the original Raw object
raw.add_channels([new_raw_array])

fif_name = os.path.join(dat_path, 'Sub001_Extern_MedOn_StimOff.fif')
raw.save(fif_name, overwrite = True)

raw = mne.io.read_raw_fif(os.path.join(
    'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Extern_Trial',
        'Sub001_Extern_MedOn_StimOn_L2.fif'
    )
)

### Add Directories/ Load Files

In [ ]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

In [ ]:
raw = mne.io.read_raw_fif(os.path.join(
    project_path,
        'data',
        'Extern_Trial',
        'raw_dat',
        'Sub001_Extern_MedOn_StimOn_L2.fif'
    )
)

In [ ]:
raw.ch_names

In [ ]:
raw.get_data(picks = 'LFPL01STN')[0].shape

In [ ]:
%matplotlib qt
plt.plot(raw['Time_Sec'][1], raw.get_data(picks = 'LFPL01STN')[0])
plt.show()

In [ ]:
raw.plot()

In [ ]:
# Filter the list to select only elements containing 'LFPL'
chans = [s for s in raw.ch_names if 'LFP' in s]
#chans = ['LFPL01STN', 'LFPL02STN', 'LFPL03STN', 'LFPL04STN', 'LFPL08STN']
print(chans)

In [ ]:
%matplotlib qt
from scipy import signal
DETREND = 1
NOTCH = 1

for ch in chans:
    dat = raw.get_data(picks = ch)[0]
    fs = raw.info['sfreq']
    stimulation_frequencies = [260, 390]
    Q = 30  # Q factor for notch filter
    window_length = int(fs)  # 1-second window length
    overlap = window_length // 4  # 25% overlap

    # Detrend the LFP signal
    if DETREND == 1:
        dat = signal.detrend(dat)

    # Apply notch filters for each frequency and its harmonics
    if NOTCH == 1:
        filtered_signals = []
        for freq in stimulation_frequencies:
            w0 = freq / (fs / 2)  # Normalized notch frequency
            b, a = signal.iirnotch(w0, Q)
            filtered_signal = signal.filtfilt(b, a, dat)
            filtered_signals.append(filtered_signal)
            # Combine the filtered signals
            dat = np.sum(filtered_signals, axis=0)

    # Calculate the short-time Fourier transform (STFT) using Hamming window
    frequencies, times, Zxx = signal.stft(dat, fs=fs, nperseg=window_length, noverlap=overlap, window='hamming')

    # Plot the spectral amplitudes
    fig, axs = plt.subplots(1, 2)
    for ax in axs.flat:
        ax.pcolormesh(times, frequencies, 20 * np.log10(np.abs(Zxx)), shading='auto', cmap='inferno', vmin=-200, vmax = -60)
        ax.set_yticks(np.arange(0, 512, 30))
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Frequency (Hz)')
        #plt.colorbar(label='Amplitude (dB)')
    
    axs[0].set_ylim(1, 100)
    axs[1].set_ylim(1, 512)

    plt.show()
    break
    #plt.savefig(os.path.join(dat_path, f'{ch}_StimOff_FFT'), dpi = 200)
